In [1]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [2]:

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!virtualenv /content/drive/MyDrive/colab_env_2

created virtual environment CPython3.11.13.final.0-64 in 16687ms
  creator CPython3Posix(dest=/content/drive/MyDrive/colab_env_2, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.9.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
pip install virtualenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 26.8 MB/s eta 0:00:00


In [4]:
!source /content/drive/MyDrive/colab_env_2/bin/activate

In [5]:
import sys
sys.path.append("/content/drive/MyDrive/colab_env_2/lib/python3.10/site-packages")

In [ ]:
!pip install yfinance pandas google-generativeai


In [6]:
from dotenv import load_dotenv


In [7]:
load_dotenv('/content/drive/MyDrive/Env/.env')

True

In [8]:

import os
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")


In [10]:
!pip install langchain langchain_groq yfinance pandas pandas-ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 22.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully

In [13]:
import os
import yfinance as yf
import pandas as pd
from langchain_groq import ChatGroq

# --- Configuration ---
# Replace "YOUR_GROQ_API_KEY" with your actual key.
# IMPORTANT: Do not share your notebook publicly with the key hardcoded.
GROQ_API_KEY = os.environ['GROQ_API_KEY']

# Check if the key has been set
if GROQ_API_KEY == "YOUR_GROQ_API_KEY" or not GROQ_API_KEY:
    print(" WARNING: Please replace 'YOUR_GROQ_API_KEY' with your actual Groq API key.")
else:
    print(" Groq API Key is set.")

# Initialize the Groq LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"
)

print(" Libraries imported and LLM client initialized successfully.")

 Groq API Key is set.
 Libraries imported and LLM client initialized successfully.


In [10]:
import yfinance as yf
import pandas as pd

def get_stock_data(ticker_symbol: str):
    """
    Fetches historical price data and fundamental company info for a given stock ticker.
    Returns a tuple: (price_history_df, fundamentals_dict)
    """
    print(f"Fetching data for ticker: {ticker_symbol}...")
    try:
        stock = yf.Ticker(ticker_symbol)

        # The .info dictionary can be large and sometimes keys are missing.
        # Accessing it can sometimes fail for certain tickers, so we wrap it.
        info = stock.info

        # Get historical market data for the last year
        hist_df = stock.history(period="1y")

        if hist_df.empty:
            print(f"Warning: No historical price data found for {ticker_symbol}. The ticker may be invalid.")
            return None, None

        # Select key fundamental metrics we care about.
        # We use .get() to avoid errors if a key is missing.
        fundamentals = {
            "company_name": info.get("longName"),
            "ticker": info.get("symbol"),
            "sector": info.get("sector"),
            "industry": info.get("industry"),
            "market_cap": info.get("marketCap"),
            "pe_ratio": info.get("trailingPE"),
            "forward_pe": info.get("forwardPE"),
            "eps": info.get("trailingEps"),
            "price_to_book": info.get("priceToBook"),
            "debt_to_equity": info.get("debtToEquity"),
            "return_on_equity": info.get("returnOnEquity"),
            "dividend_yield": info.get("dividendYield"),
            "52_week_high": info.get("fiftyTwoWeekHigh"),
            "52_week_low": info.get("fiftyTwoWeekLow"),
            "average_volume": info.get("averageVolume"),
        }

        print(f"Successfully fetched data for {fundamentals.get('company_name', ticker_symbol)}.")
        return hist_df, fundamentals

    except Exception as e:
        print(f"An error occurred while fetching data for {ticker_symbol}: {e}")
        return None, None

# --- Let's test the function for Apple Inc. ---
ticker = "AAPL"
price_history, fundamental_data = get_stock_data(ticker)

# --- Verify the output ---
if price_history is not None and fundamental_data is not None:
    print("\n--- Fundamental Data Sample ---")
    # Print a few key-value pairs from the fundamentals dictionary
    for key, value in list(fundamental_data.items())[:5]:
        print(f"{key.replace('_', ' ').title()}: {value}")

    print("\n--- Recent Price History Sample ---")
    # Print the last 5 rows of the price history dataframe
    print(price_history.tail())
else:
    print("\nData fetching failed. Please check the ticker symbol or your network connection.")

Fetching data for ticker: AAPL...
Successfully fetched data for Apple Inc..

--- Fundamental Data Sample ---
Company Name: Apple Inc.
Ticker: AAPL
Sector: Technology
Industry: Consumer Electronics
Market Cap: 3473692426240

--- Recent Price History Sample ---
                                 Open        High         Low       Close  \
Date                                                                        
2025-09-08 00:00:00-04:00  239.300003  240.149994  236.339996  237.880005   
2025-09-09 00:00:00-04:00  237.000000  238.779999  233.360001  234.350006   
2025-09-10 00:00:00-04:00  232.190002  232.419998  225.949997  226.789993   
2025-09-11 00:00:00-04:00  226.880005  230.449997  226.649994  230.029999   
2025-09-12 00:00:00-04:00  229.220001  234.509995  229.020004  234.070007   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-09-08 00:00:00-04:00  48999500        0.0           0.0  
2025-09-09 

In [11]:
import pandas_ta as ta

def perform_technical_analysis(df: pd.DataFrame):
    """
    Calculates key technical indicators and adds them to the price history DataFrame.
    """
    print("Performing technical analysis...")
    # Make a copy to avoid modifying the original DataFrame unexpectedly
    df_ta = df.copy()

    # Calculate 50-day and 200-day Simple Moving Averages
    df_ta.ta.sma(length=50, append=True)
    df_ta.ta.sma(length=200, append=True)

    # Calculate Relative Strength Index (RSI)
    df_ta.ta.rsi(length=14, append=True)

    # Calculate Moving Average Convergence Divergence (MACD)
    # This adds three columns: MACD, MACD Histogram, and MACD Signal
    df_ta.ta.macd(fast=12, slow=26, signal=9, append=True)

    # Clean up the default column names for better readability
    df_ta.rename(columns={
        "SMA_50": "50_day_sma",
        "SMA_200": "200_day_sma",
        "RSI_14": "rsi",
        "MACD_12_26_9": "macd",
        "MACDh_12_26_9": "macd_histogram",
        "MACDs_12_26_9": "macd_signal"
    }, inplace=True)

    print("Technical analysis complete.")
    return df_ta

# --- Run the analysis on the data we fetched in Step 2 ---
# Ensure you have the 'price_history' variable from the previous step
if 'price_history' in locals() and price_history is not None:
    price_history_with_ta = perform_technical_analysis(price_history)

    # --- Verify the output ---
    print("\n--- Price History with new Technical Indicator Columns ---")
    # Display the last 5 rows. You should see new columns on the right.
    print(price_history_with_ta.tail())
else:
    print("Could not find 'price_history' data. Please re-run Step 2 successfully first.")

Performing technical analysis...
Technical analysis complete.

--- Price History with new Technical Indicator Columns ---
                                 Open        High         Low       Close  \
Date                                                                        
2025-09-08 00:00:00-04:00  239.300003  240.149994  236.339996  237.880005   
2025-09-09 00:00:00-04:00  237.000000  238.779999  233.360001  234.350006   
2025-09-10 00:00:00-04:00  232.190002  232.419998  225.949997  226.789993   
2025-09-11 00:00:00-04:00  226.880005  230.449997  226.649994  230.029999   
2025-09-12 00:00:00-04:00  229.220001  234.509995  229.020004  234.070007   

                             Volume  Dividends  Stock Splits  50_day_sma  \
Date                                                                       
2025-09-08 00:00:00-04:00  48999500        0.0           0.0  219.007781   
2025-09-09 00:00:00-04:00  66313900        0.0           0.0  219.677740   
2025-09-10 00:00:00-04:00  8344080

In [14]:
from langchain_core.prompts import ChatPromptTemplate

def get_llm_summary(ticker_symbol: str, fundamentals: dict, latest_ta_df: pd.DataFrame):
    """
    Uses the LLM to generate a financial summary based on provided data.
    """
    print("Generating financial summary with Groq LLM...")

    # Create a structured prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert financial analyst. Your role is to synthesize raw financial data into a clear, concise, and insightful summary for a retail investor. Do not give financial advice or make future predictions. Focus on summarizing the provided data objectively.",
            ),
            (
                "human",
                """
                Please analyze the following financial data for the stock with ticker symbol: {ticker}.

                **Fundamental Data:**
                - Company Name: {company_name}
                - Sector: {sector}
                - Market Cap: {market_cap}
                - P/E Ratio: {pe_ratio}
                - Forward P/E Ratio: {forward_pe}
                - EPS (Earnings Per Share): {eps}
                - Debt-to-Equity Ratio: {debt_to_equity}
                - Return on Equity: {return_on_equity}
                - Dividend Yield: {dividend_yield}

                **Latest Technical Indicators:**
                - Current Price: {current_price}
                - 50-Day Moving Average: {fifty_day_sma}
                - 200-Day Moving Average: {two_hundred_day_sma}
                - RSI (Relative Strength Index): {rsi}

                **Your Task:**
                Provide a summary of the company's financial health and market position based *only* on the data above. Structure your response into two distinct paragraphs:
                1.  **Fundamental Analysis:** Briefly describe the company's valuation and financial stability using the fundamental metrics.
                2.  **Technical Analysis:** Describe the stock's recent price trend and momentum based on the technical indicators. For RSI, a value over 70 is often considered overbought, and under 30 is oversold.
                """
            ),
        ]
    )

    # Combine the prompt and the LLM into a runnable chain
    chain = prompt | llm

    # Get the very last row of technical analysis data
    latest_indicators = latest_ta_df.iloc[-1]

    # Invoke the chain with the necessary data.
    # We use .get() and provide a default value like 'N/A' to prevent errors
    # if a piece of data is missing from the source.
    response = chain.invoke({
        "ticker": ticker_symbol,
        "company_name": fundamentals.get('company_name', 'N/A'),
        "sector": fundamentals.get('sector', 'N/A'),
        "market_cap": f"${fundamentals.get('market_cap', 0):,}",
        "pe_ratio": f"{fundamentals.get('pe_ratio', 0):.2f}",
        "forward_pe": f"{fundamentals.get('forward_pe', 0):.2f}",
        "eps": f"{fundamentals.get('eps', 0):.2f}",
        "debt_to_equity": f"{fundamentals.get('debt_to_equity', 0):.2f}",
        "return_on_equity": f"{fundamentals.get('return_on_equity', 0):.2%}",
        "dividend_yield": f"{fundamentals.get('dividend_yield', 0):.2%}",
        "current_price": f"${latest_indicators.get('Close', 0):.2f}",
        "fifty_day_sma": f"${latest_indicators.get('50_day_sma', 0):.2f}",
        "two_hundred_day_sma": f"${latest_indicators.get('200_day_sma', 0):.2f}",
        "rsi": f"{latest_indicators.get('rsi', 0):.2f}",
    })

    print("LLM summary generated successfully.")
    return response.content

# --- Generate the summary using variables from previous steps ---
if 'fundamental_data' in locals() and 'price_history_with_ta' in locals():
    summary = get_llm_summary(ticker, fundamental_data, price_history_with_ta)

    print(f"\n--- LLM-Powered Financial Summary for {ticker} ---")
    print(summary)
else:
    print("Could not find necessary data. Please re-run Steps 2 and 3 successfully.")

Generating financial summary with Groq LLM...
LLM summary generated successfully.

--- LLM-Powered Financial Summary for AAPL ---
**Fundamental Analysis:**
Apple Inc., with a market capitalization of over $3.4 trillion, is a large-cap company in the technology sector. The company's valuation can be assessed through its P/E ratio of 35.47, which is higher than its forward P/E ratio of 28.17, indicating that investors may have high expectations for the company's future growth. The EPS of $6.60 suggests a stable earnings performance. However, the debt-to-equity ratio of 154.49 is notably high, which could indicate a significant reliance on debt financing. On the other hand, the return on equity of 149.81% is exceptionally high, suggesting that the company is highly efficient in generating profits from its shareholders' equity. The dividend yield of 44.00% appears unusually high and may be a result of an error in the data, as typical dividend yields for established companies like Apple are

In [15]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# 1. Define the desired structured output for our recommendation
class StockRecommendation(BaseModel):
    """A structured recommendation for a stock."""
    recommendation: str = Field(description="The final recommendation, which must be one of: 'Buy', 'Hold', or 'Sell'.")
    confidence_score: float = Field(description="A confidence score from 0.0 to 1.0 for the recommendation, where 1.0 is highest confidence.")
    explanation: str = Field(description="A detailed, multi-paragraph explanation supporting the recommendation, synthesizing all provided data.")

# 2. Create the main orchestrator function
def generate_structured_recommendation(ticker_symbol: str):
    """
    Orchestrates the entire workflow:
    1. Fetches data.
    2. Performs technical analysis.
    3. Generates a structured recommendation using an LLM.
    """
    print(f"--- Starting Full Analysis for {ticker_symbol} ---")

    # Step 1: Fetch data (using our function from Step 2)
    prices, fundamentals = get_stock_data(ticker_symbol)
    if prices is None:
        print(f"Analysis stopped: Could not fetch data for {ticker_symbol}.")
        return None

    # Step 2: Perform technical analysis (using our function from Step 3)
    prices_with_ta = perform_technical_analysis(prices)
    latest_indicators = prices_with_ta.iloc[-1]

    # Step 3: Generate the structured recommendation with the LLM
    print("Generating structured recommendation with Groq LLM...")

    # Create a new LLM instance that is specifically configured to produce structured output
    structured_llm = llm.with_structured_output(StockRecommendation)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a senior investment analyst. Your task is to provide a clear, data-driven investment recommendation (Buy, Hold, or Sell) with a confidence score. You must base your entire analysis and explanation *only* on the data provided. Do not use any external knowledge."),
            ("human", """
                Analyze the stock: {ticker}

                **Fundamental Data:**
                {fundamentals}

                **Technical Analysis Data (Latest):**
                - Current Price: {current_price}
                - 50-Day SMA: {fifty_day_sma}
                - 200-Day SMA: {two_hundred_day_sma}
                - RSI: {rsi}
                - MACD Line: {macd}
                - MACD Signal Line: {macd_signal}

                **Your Task:**
                Based on a holistic analysis of the provided fundamental and technical data, generate a structured recommendation.
                - **For Fundamentals:** Consider if the P/E ratio suggests the stock is over or undervalued compared to typical market values. Look at debt, profitability (Return on Equity), and dividend yield for stability.
                - **For Technicals:** Is the current price above or below key moving averages (a key trend indicator)? Is the RSI indicating overbought (>70) or oversold (<30) conditions? Is the MACD line above its signal line (a bullish sign) or below (a bearish sign)?
                - **Synthesize:** Combine these insights into a final recommendation, a confidence score, and a detailed explanation that justifies your reasoning.
            """)
        ]
    )

    chain = prompt | structured_llm

    recommendation_object = chain.invoke({
        "ticker": ticker_symbol,
        "fundamentals": str(fundamentals), # Pass the whole dict as a string for the LLM to read
        "current_price": f"${latest_indicators.get('Close', 0):.2f}",
        "fifty_day_sma": f"${latest_indicators.get('50_day_sma', 0):.2f}",
        "two_hundred_day_sma": f"${latest_indicators.get('200_day_sma', 0):.2f}",
        "rsi": f"{latest_indicators.get('rsi', 0):.2f}",
        "macd": f"{latest_indicators.get('macd', 0):.2f}",
        "macd_signal": f"{latest_indicators.get('macd_signal', 0):.2f}"
    })

    print("--- Analysis Complete ---")
    return recommendation_object

# --- Run the full workflow for a different stock, e.g., NVIDIA ---
nvda_recommendation = generate_structured_recommendation("NVDA")

# --- Verify the output ---
if nvda_recommendation:
    print("\n--- Final Structured Recommendation for NVDA ---")
    print(f"Recommendation: {nvda_recommendation.recommendation}")
    print(f"Confidence Score: {nvda_recommendation.confidence_score:.2f}")
    print("\n--- Explanation ---")
    print(nvda_recommendation.explanation)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


--- Starting Full Analysis for NVDA ---
Fetching data for ticker: NVDA...
Successfully fetched data for NVIDIA Corporation.
Performing technical analysis...
Technical analysis complete.
Generating structured recommendation with Groq LLM...
--- Analysis Complete ---

--- Final Structured Recommendation for NVDA ---
Recommendation: Buy
Confidence Score: 0.80

--- Explanation ---
The stock NVDA has a high P/E ratio of 50.805717, which may indicate overvaluation. However, the forward P/E ratio is slightly lower at 43.1602, suggesting some potential for growth. The debt-to-equity ratio is relatively low at 10.584, and the return on equity is 1.09417, indicating stable profitability. The dividend yield is low at 0.02, which may not be attractive to income investors. On the technical side, the current price of $177.82 is above the 50-Day SMA of $174.11 and the 200-Day SMA of $139.95, indicating a positive trend. The RSI of 55.53 is within the neutral range, and the MACD line is slightly above